### Exercises

#### Question 1

We want to write a function that can find an approximate maximum or minimum of some given function over some given range.

For example, given some function:

```
f(x) = x**2 - 1
```

our function should return an approximate minimum (or maximum) of `f` over some given range, say `[-5, 5]`.

We'll do this by essentially splitting our intervals into `n` points (what I'll call the `resolution`), evaluating the function at each of these points, and returning either the min or the max.

We want this function to be generic, so it should have the following parameters:
- a function of one variable
- a range of values defined by start/end values
- a value indicating the "resolution"
- a value indicating whether we want the min or the max

In [20]:
import numpy as np


f1 = lambda x: x ** 2 - 1
f2 = lambda x: abs(x-2)
f3 = lambda x: np.sin(x)



def find_min_max(func, start=-10, end=10, resolution=1_000, is_min=True):
    range_ = np.linspace(start, end, resolution)
    return np.min(func(range_)) if is_min else np.max(func(range_))


In [21]:
find_min_max(f3, is_min=False)

0.9999996994977832

#### Question 2

You are given a function of two variables, and a list of tuples containing the values for the two variables.

Create a list that is the result of calling the function on each values in the list, using three different techniques:
- a `for` loop
- a list comprehension
- the `map` function

Use the `timeit` function to time each approach.

Hint: write a function that implements each approach, and then time calling those functions using the `timeit` function (`from timeit import timeit` - we've used it before). Also you will want to specify `number=10` or something like that when you run `timeit` - unless you want to sit there watvhing your screen for quite a while :-)

In [22]:
import math

def func(point):
    # expect point to be a sequence of two values
    x, y = point
    return math.hypot(x, y)  
    # hypot is a function that calculates sqrt(x**2 + y**2), given a sequence (x, y)

points = [
    (0, 0),
    (1, 1),
    (10, 20),
    (math.pi, math.e)
]

Your result for `points` should be:

```
[0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313314]
```

In [29]:
# method 1
from timeit import timeit

def calc_loop(f, points):
    results = []
    for p in points:
        results.append(f(p))
    return results

calc_loop(func, points)


[0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]

In [30]:
def calc_comprehension(f, points):
    return [f(p) for p in points]

calc_comprehension(func, points)
    

[0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]

In [31]:
def calc_map(f, points):
    return list(map(f, points))

calc_map(func, points)

[0.0, 1.4142135623730951, 22.360679774997898, 4.154354402313313]

Now let's run some timings, using `points_large` for our arguments:

In [2]:
points_large = [(math.sin(x), math.cos(x)) for x in range(1, 1_000_000)]

In [33]:
timeit("calc_loop(func, points)", globals=globals(), number=10)

2.729997504502535e-05

In [35]:
timeit("calc_comprehension(func, points)", globals=globals(), number=10)

2.6699970476329327e-05

In [36]:
timeit("calc_map(func, points)", globals=globals(), number=10)

2.6699970476329327e-05

#### Question 3

Write a function that returns a function with all arguments, except the first one, prefilled with certain values provided to the outer function.

(This is sometimes called a partial function).

For example, we may have some functions such as:

In [3]:
import math

In [4]:
def power(x, n):
    return x ** n

In [5]:
def dist(pt1, pt2):
    return math.sqrt(sum(coord_1 - coord_2 for coord_1, coord_2 in zip(pt1, pt2)))

Or even functions already defined, such as:

```
math.gcd(a, b)
```
or
```
math.log(x, base)
```

We want to to be able to generate new functions, based on these ones (`power`, `dist`, `gcd`, `log`) but with all the values except the first one prefilled, for example, assuming our function is named `partial`, we can use it to define new functions this way:

```
squares = partial(power, 2)
dist_from_origin = partial(dist, (0, 0))
gcd_13 = partial(math.gcd, 13)
log_2 = partial(math.log, 2)
log_10 = partial(math.log, 10)
log_16 = partial(math.log, 16)
```

Then when we call our new functions, we just pass in the value for the first argument, i.e.

```
squares(3) --> 9
squares(4) --> 16
dist_from_origin((1, 1)) --> 1.414
log_2(10) --> 3.3219
log_10(10) --> 1.0
log_16(10) --> 0.8304
```

In [42]:
def power(x, n):
    return x ** n

def partial(func, *args, **kwargs):
    def inner(first_arg):
        print("func: ", func.__name__)
        print("first_arg: ", first_arg)
        print("args: ", args)
        print("kwargs: ", kwargs)
        return func(first_arg, *args, **kwargs)
    return inner

In [43]:
f = partial(power, n=2)

In [44]:
f(3)

func:  power
first_arg:  3
args:  ()
kwargs:  {'n': 2}


9

#### Question 4

Write a function that can be used to not only execute another function with specified arguments, but print a "log" (basically just print to the console", of how long it took to execute the function.

For example, given some functions like this:

In [45]:
def norm(x, y):
    return math.sqrt(x**2 + y**2)

def find_index_min(seq):
    min_ = min(seq)
    return seq.index(min_)

Then assuming your logging function is called `logged`, you could create logged functions this way:

In [47]:
from time import perf_counter

def logged(f):
    def inner(*args, **kwargs):
        start = perf_counter()
        result = f(*args, **kwargs)
        end = perf_counter()
        print(f"Function {f.__name__} took {end - start} seconds")
        return result
    return inner

In [48]:
norm_logged = logged(norm)
find_index_min_logged = logged(find_index_min)

You would then be able to call `norm_logged` with some arguments, or `find_index_min_logged` with some arguments, and not only get the actual result back, but also see an output to the console that tells you how long the function took to run.

In [49]:
norm_logged(3, 4)

Function norm took 0.0006563999922946095 seconds


5.0

In [55]:
@logged
def norm(x, y):
    return math.sqrt(x**2 + y**2)

norm(3, 4)

Function norm took 5.89998671784997e-06 seconds


5.0

In [51]:
norm_logged.__closure__

(<cell at 0x000001224B16A680: function object at 0x000001224AFD4400>,)

In [54]:
hex(id(norm))

'0x1224afd4400'